In [1]:
import jax
import jax.numpy as jnp

import mfvi

from typing import NamedTuple, Union, Iterable, Mapping, Any
from optax import OptState

In [2]:
mu = jnp.zeros((1,))
rho = jnp.ones((1,))
sample = jnp.ones((1,))
params = mu, rho
meanfield_logprob = mfvi.meanfield_logprob(params, sample)
print(meanfield_logprob)

-1.9866061


In [13]:
mu = jnp.zeros((1,))
rho = jnp.ones((1,))
sample = jnp.zeros((1,))

params = mu, rho
sampled_params, new_key = mfvi.meanfield_sample(params, jax.random.PRNGKey(1), 2)

In [14]:
meanfield_logprob = mfvi.meanfield_logprob(params, sampled_params)
meanfield_logprob

Array(-4.55315, dtype=float32)

In [22]:
observed = jax.random.normal(jax.random.PRNGKey(1), shape=(100,)) + 10 * 2
def logdensity_fn(x):
    logpdf = jax.scipy.stats.norm.logpdf(observed, x[0], x[1])
    return jnp.sum(logpdf)

In [23]:
mfvi.meanfield_elbo(
    params,
    jax.random.PRNGKey(2),
    logdensity_fn, 
    5
)

-2.183433 -27209.63


(Array(27207.447, dtype=float32),
 Array([ 794799940, 3892463417], dtype=uint32))